In [ ]:
## ensure necessary libraries are active
library(tidyverse)
library(sqldf)
library(lmtest)
library(sandwich)
print("Complete!!")

In [ ]:
## data set included for population by counties, 2006 - 2014
## data set from ARCOS data
countypop = read.csv("/kaggle/input/arcos-wapo-api-data/data/pop_counties_20062014.csv")
head(countypop, 2)

In [ ]:
## data for doses and shipments per county, 2006 - 2014
## data set from ARCOS data
county14 = read.csv("/kaggle/input/arcos-wapo-api-data/data/county_annual14.csv")
head(county14, 2)

In [ ]:
## add data on the population of each county receiving SNAP benefits
## data from Census.gov SAIPE datasets
snapdata = read.csv("/kaggle/input/snap-beneficiaries-by-fips-by-year-2006-2014/FIPSsnapbenefits - fips_year_snap.csv")
head(snapdata, 2)
snapdata = snapdata %>%
          select(c(1,3,4)) %>%
        rename("snap_pop" = "X",
              "countyfips" = "fips")
head(snapdata, 2)
### ensure everything looks right / make it workable
unique(county14$year)
nrow(county14)
class(snapdata$snap_pop)
snap2 = snapdata
snap2$snap_pop <- as.numeric(gsub(",","",snap2$snap_pop))
head(snap2, 2)
class(snap2$snap_pop)


In [ ]:
### merge doses and population data
### some records are deleted, due to a lack of data in both sets
popdose = inner_join(county14, countypop, 
                  by = c('countyfips', 'year'))
### select relevant columns and clean names
popdosedf = popdose %>%
    select(c(1,2,3,5,6,14))
popdosedf <- popdosedf %>% 
        rename("county" ="BUYER_COUNTY.x",
               "state" = "BUYER_STATE.x",
               "doses" = "DOSAGE_UNIT")
### add a column measuring pills per capita
popdosedf$pills_per_capita = round(popdosedf$doses / popdosedf$population, 2)
### ensure data integrity / "make sure it looks right"
head(popdosedf)
print(paste("Number of observations in this data set: ", nrow(popdosedf)))

In [ ]:
## combine dose, population, and snap data
## create per capita measures for pills and snap recipients
popdosesnap = inner_join(popdosedf, snap2, 
                  by = c('countyfips', 'year'))
popdosesnap$snap_per_capita = round(popdosesnap$snap_pop / popdosesnap$population, 2)
head(popdosesnap)
print(paste("Number of observation in this data set: ", nrow(popdosesnap)))

In [ ]:
### import info on pharmacy / buyer types and locations
buyerannual = read.csv("/kaggle/input/arcos-wapo-api-data/data/buyer_annual14.csv")
pharmfips = read.csv("/kaggle/input/arcos-wapo-api-data/data/pharmacies_counties14.csv")

In [ ]:
### merge and clean pharmacy and dosage type into a single dataframe
pharmdose = inner_join(buyerannual, pharmfips, 
                  by = 'BUYER_DEA_NO')
pharmdosedf = pharmdose %>%
    select(c(1,2,5,6,11)) %>%
    rename("countyfips" = "county_fips")
head(pharmdosedf)
### number of chain vs retail (independent) pharmacies
table(pharmdosedf$BUYER_BUS_ACT)

In [ ]:
### boxplot with outliers
ggplot(data=pharmdosedf, aes(x=BUYER_BUS_ACT, y=DOSAGE_UNIT)) +
    geom_boxplot() + 
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Opioids Doses", x = "Pharmacy Type",
         title = "Figure 1A: Distribution of Doses Based on Pharmacy Type")

In [ ]:
### boxplot without outliers
ggplot(data=pharmdosedf, aes(x=BUYER_BUS_ACT, y=DOSAGE_UNIT)) +
    geom_boxplot(outlier.alpha = 0) + 
    scale_y_continuous(limits = quantile(pharmdosedf$DOSAGE_UNIT, c(0.1, 0.9)))+
    stat_summary(fun.y=mean, geom="point", shape=2, size=4) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Opioids Doses", x = "Pharmacy Type",
         title = "Figure 1B: Distribution of Doses Based on Pharmacy Type without Outliers")

In [ ]:
### boolean code for independent pharmacies
pharmdosedf$is_independent = ifelse(pharmdosedf$BUYER_BUS_ACT == 'RETAIL PHARMACY', 1, 0)

In [ ]:
### create a dataframe to merge independent percent of pharmacies
sqlinddf = sqldf("SELECT countyfips, year, AVG(is_independent) AS ind_perc
    FROM pharmdosedf
    GROUP BY countyfips, YEAR")

In [ ]:
### create dataframe with the number of pharmacies by county
sqlrxcount = sqldf("SELECT countyfips, year, COUNT(BUYER_BUS_ACT) AS pharmacies
    FROM pharmdosedf
    GROUP BY countyfips, YEAR")

In [ ]:
### creating a finalized dataframe
finalerdf = inner_join(popdosesnap, sqlinddf,
                    by = c('countyfips', 'year'))
finalerdf = inner_join(finalerdf, sqlrxcount,
                    by = c('countyfips', 'year'))
head(finalerdf)
nrow(finalerdf)
colSums(is.na(finalerdf))
### drop 2 incomplete records
finalerdf = na.omit(finalerdf)
nrow(finalerdf)

In [ ]:
### plot pharmacies and population
ggplot(data = finalerdf, aes(x=pharmacies, y=population)) +
    geom_point(alpha=.3) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Population", x = "Number of Pharmacies",
         title = "Figure 2: Number of Pharmacies and Population")

In [ ]:
### plot independent percent and population
ggplot(data = finalerdf, aes(x=ind_perc, y=population)) +
    geom_point(alpha=.3) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Population", x = "Ratio of Pharmacies in a County which are Independent",
         title = "Figure 3: Independent Pharmacy Percentage and Population")

In [ ]:
### plot pharmacies and independent ratio
ggplot(data = finalerdf, aes(y=pharmacies, x=ind_perc)) +
    geom_point(alpha=.3) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(x = "Ratio of Independent Pharmacies", y = "Number of Pharmacies",
         title = "Figure 4: Number of Pharmacies and Ratio of Independent Pharmacies")

In [ ]:
### plot pharmacies and opioids per capita
ggplot(data = finalerdf, aes(y=pharmacies, x=pills_per_capita)) +
    geom_point(alpha=.3) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(x = "Opioid Pills per Capita", y = "Number of Pharmacies",
         title = "Figure 5: Number of Pharmacies and Opioids per Capita")

In [ ]:
### plot independent ratio and opioids per capita
ggplot(data = finalerdf, aes(x=ind_perc, y=pills_per_capita)) +
    geom_point(alpha=.3) +
    geom_smooth(method='lm', se=FALSE) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Opioid Pills per Capita", x = "Ratio of Pharmacies in a County which are Independent",
         title = "Figure 6: Independent Pharmacy Percentage and Opioids per Capita")

In [ ]:
### plot population and opioids per capita
ggplot(data = finalerdf, aes(y=population, x=pills_per_capita)) +
    geom_point(alpha=.3) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(x = "Opioid Pills per Capita", y = "County Population",
         title = "Figure 7: County Population and Opioids per Capita")

In [ ]:
### plot pharmacies and SNAP recipients
ggplot(data = finalerdf, aes(y=pharmacies, x=snap_per_capita)) +
    geom_point(alpha=.3) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(x = "SNAP Recipients per Capita", y = "Number of Pharmacies",
         title = "Figure 8: Number of Pharmacies and SNAP Recipients")

In [ ]:
### plot population and SNAP recipients
ggplot(data = finalerdf, aes(y=population, x=snap_per_capita)) +
    geom_point(alpha=.3) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(x = "SNAP Recipients per Capita", y = "County Population",
         title = "Figure 9: Population and SNAP Recipients")

In [ ]:
### plot opioids and SNAP recipients, both per capita
ggplot(data = finalerdf, aes(x=snap_per_capita, y=pills_per_capita)) +
    geom_point(alpha=.3) +
    geom_smooth(method = 'lm', se=FALSE) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Opioid Pills per Capita", x = "SNAP Recipients per Capita",
         title = "Figure 10: Per Capita Relation of Opioids and SNAP Recipients")

In [ ]:
### plot opioids and SNAP recipients, per capita, by year
ggplot(data = finalerdf, aes(x=snap_per_capita, y=pills_per_capita)) +
    geom_point(alpha=.3) +
    geom_smooth(method = 'lm', se=FALSE) +
    facet_wrap(~year) +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Opioid Pills per Capita", x = "SNAP Recipients per Capita",
         title = "Figure 10B: Per Capita Relation of Opioids and SNAP Recipients by Year")

In [ ]:
### explore the average pills per capita by year, across all counties
querypopdose = sqldf("SELECT year, AVG(pills_per_capita) AS average_pills_per_cap
                FROM popdosedf
                GROUP BY year")
ggplot(querypopdose, aes(x=year, y = average_pills_per_cap)) +
    geom_line() +
    theme(panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          axis.line = element_line(colour = "black")) +
    labs(y = "Average Pills per Capita", x = "Year",
         title = "Figure 11: Average Pills per Capita in the US by Year")

In [ ]:
### summary statistics for data set
summary(finalerdf[,c(4,6:11)])

In [ ]:
### create a basic regression model
basicmodel = lm(pills_per_capita ~ snap_per_capita, data=finalerdf)
summary(basicmodel)

In [ ]:
#create residual vs. fitted plot
plot(fitted(basicmodel), resid(basicmodel), xlab='Fitted Values', ylab='Residuals')

#add a horizontal line at 0 
abline(0,0)

In [ ]:
#perform Breusch-Pagan test
bptest(basicmodel)

#perform Durbin-Watson test
dwtest(formula = basicmodel,  alternative = "two.sided")

In [ ]:
### run basic regression with robust standard errors
coeftest(basicmodel, vcov = vcovHC(basicmodel, type = 'HC0'))

In [ ]:
#define weights to use
wt <- 1 / lm(abs(basicmodel$residuals) ~ basicmodel$fitted.values)$fitted.values^2

#perform weighted least squares regression
wls_model <- lm(pills_per_capita ~ snap_per_capita, data = finalerdf, weights = wt)

#view summary of model
summary(wls_model)

In [ ]:
### snap per capita with fixed effects of year
summary(lm(pills_per_capita ~ snap_per_capita + factor(year),
   data = finalerdf))

In [ ]:
head(finalerdf,1)

In [ ]:
### kitchen sink model; add everything 
ksmodel = lm(pills_per_capita ~ snap_per_capita + ind_perc + pharmacies + population +
            year, data = finalerdf)
### ksmodel with year fixed effect
ksmodelfe = lm(pills_per_capita ~ snap_per_capita + ind_perc + pharmacies + population +
            factor(year), data = finalerdf)
### basic ks regression
summary(ksmodel)

In [ ]:
### run ksmodel with robust standard errors
coeftest(ksmodel, vcov = vcovHC(ksmodel, type = 'HC0'))

In [ ]:
### run ksmodelfe with robust standard errors
coeftest(ksmodelfe, vcov = vcovHC(ksmodelfe, type = 'HC0'))

In [ ]:
#define weights to use
kswt <- 1 / lm(abs(ksmodel$residuals) ~ ksmodel$fitted.values)$fitted.values^2

#perform weighted least squares regression
ks_wls_model <- lm(pills_per_capita ~ snap_per_capita + ind_perc +
                   pharmacies + population + year,
                   data = finalerdf, weights = kswt)

#view summary of model
summary(ks_wls_model)

In [ ]:
#define weights to use
ksfewt <- 1 / lm(abs(ksmodelfe$residuals) ~ ksmodelfe$fitted.values)$fitted.values^2

#perform weighted least squares regression
ksfe_wls_model <- lm(pills_per_capita ~ snap_per_capita + ind_perc +
                   pharmacies + population + factor(year),
                   data = finalerdf, weights = ksfewt)

#view summary of model
summary(ksfe_wls_model)

In [ ]:
summary(lm(pills_per_capita ~ snap_per_capita + ind_perc + year^2 +
           countyfips + 1/pharmacies + 1/population,
           data = finalerdf))